In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.callbacks import EarlyStopping
from kerastuner.tuners import RandomSearch
from tensorflow import keras
import random
from keras.optimizers import Adam

# Seed value
seed_value = 42
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Load data
data = pd.read_csv('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/Final_df/BTC_FINAL_DF.csv')


# Make the date the index
data.set_index('Date', inplace=True)

# List of columns to scale
columns_to_scale = ['Open', 'High', 'Low', 'Close','Volume', 'RSI', 'ATR', 'MACD', 'MFI',
                    'EMA', 'SMA', 'OBV', 'GTrends_Interest', 'Sentiment_Bullish',
                    'Price_oil', 'Price_gold', 'Price_NASDAQ', 'Price_SP500', 'Price_NYSE',
                    'Interest_Rate', 'hash_rate', 'users']

# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the selected columns
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Extract the min and max values for 'Close' for manual inverse transformation
close_min = scaler.data_min_[columns_to_scale.index('Close')]
close_max = scaler.data_max_[columns_to_scale.index('Close')]

# Manually inverse transform the predictions and y_test
def manual_inverse_transform(scaled_values, min_val, max_val):
    return scaled_values * (max_val - min_val) + min_val

# Train and test data
training_size = int(len(data) * 0.9)
training_data = data[:training_size]
test_data = data[training_size:]

train_data = training_data[:int(len(training_data) * 0.9)]
val_data = training_data[int(len(training_data) * 0.9):]


# Define the function to create the dataset
def create_dataset(data, window_size, target_index):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data.iloc[i:(i + window_size)].values)  
        y.append(data.iloc[i + window_size, target_index])
    return np.array(X), np.array(y)

# Forecast Horizon
window_size = 3

# Target index ('Close')
close_index = data.columns.get_loc('Close')

# Create the dataset
X_train, y_train = create_dataset(train_data, window_size, close_index)
X_test, y_test = create_dataset(test_data, window_size, close_index)
X_val, y_val = create_dataset(val_data, window_size, close_index)
x_train_full, y_train_full = create_dataset(training_data, window_size, close_index)

# Reshape the data
X_train = X_train.reshape((X_train.shape[0], window_size, X_train.shape[2]))
X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2]))
X_val = X_val.reshape((X_val.shape[0], window_size, X_val.shape[2]))
x_train_full = x_train_full.reshape((x_train_full.shape[0], window_size, x_train_full.shape[2]))

def build_model(hp):
    model = Sequential()
    units=hp.Int('units', min_value=100, max_value=200, step=10)
    model.add(LSTM(units=units,
                    return_sequences=True, input_shape=(3, 22)))
    
    model.add(LSTM(units=units,
                    return_sequences=True))
    
    model.add(LSTM(units=units,
                   return_sequences=False))
    
    model.add(Dense(units=1, activation='linear'))

    model.compile(optimizer=Adam(
        learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='mean_squared_error',
        metrics=['mae', 'mean_absolute_percentage_error'])
    
    return model

# Try 1: LSTM-3D-REG

# Instantiate the tuner
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_percentage_error',
    max_trials=200,  
    executions_per_trial=1,  
    directory='my_dir',
    project_name='LSTM-3D-REG-2',
    overwrite=False
)

# Configure EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_mean_absolute_percentage_error',   
    patience=15,
    restore_best_weights=True 
)

# Execute the search with EarlyStopping
tuner.search(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=2,
    callbacks=[early_stopping]  
)

# Get the best model
best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)

print(best_model.summary())
learning_rate = best_model.optimizer.learning_rate.numpy()  # Ensure TensorFlow 2.x
print("Learning rate of the best model:", learning_rate)


Reloading Tuner from my_dir/LSTM-3D-REG-2/tuner0.json


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_9 (LSTM)                   │ (None, 3, 190)         │       161,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ (None, 3, 190)         │       289,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 190)            │       289,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           191 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 741,191 (2.83 MB)

 Trainable params: 741,191 (2.83 MB)

 Non-trainable params: 0 (0.00 B)

None
Learning rate of the best model: 0.01


In [3]:
# Fit the model 30 times and get average metrics for test data
num_iterations = 30
mae_unscaled_list = []
rmse_unscaled_list = []
mape_unscaled_list = []

for _ in range(num_iterations):
    best_hps = tuner.get_best_hyperparameters()[0]
    best_model = tuner.hypermodel.build(best_hps)

    history = best_model.fit(
        x_train_full, y_train_full,
        epochs=250,
        batch_size=32,
        verbose=0
    )

    predictions = best_model.predict(X_test)

    # Manually inverse transform the predictions and y_test
    predictions_unscaled = manual_inverse_transform(predictions, close_min, close_max)
    y_test_unscaled = manual_inverse_transform(y_test.reshape(-1, 1), close_min, close_max)

    # Calculate unscaled metrics
    mae_unscaled = mean_absolute_error(y_test_unscaled, predictions_unscaled)
    rmse_unscaled = np.sqrt(mean_squared_error(y_test_unscaled, predictions_unscaled))
    mape_unscaled = tf.keras.losses.MAPE(y_test_unscaled, predictions_unscaled).numpy()

    mae_unscaled_list.append(mae_unscaled)
    rmse_unscaled_list.append(rmse_unscaled)
    mape_unscaled_list.append(mape_unscaled)


average_mae_unscaled = np.mean(mae_unscaled_list)
average_rmse_unscaled = np.mean(rmse_unscaled_list)
average_mape_unscaled = np.mean(mape_unscaled_list)


# Print the average unscaled metrics
print('Average Unscaled Mean Absolute Error:', average_mae_unscaled)
print('Average Unscaled Root Mean Squared Error:', average_rmse_unscaled)
print('Average Unscaled Mean Absolute Percentage Error:', average_mape_unscaled)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


...

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Average Unscaled Mean Absolute Error: 3627.61036484796
Average Unscaled Root Mean Squared Error: 6479.342323559951
Average Unscaled Mean Absolute Percentage Error: 7.413481


In [7]:
# Seed
seed_value = 42
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)


best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)

# Train the final model 
final_model = tuner.hypermodel.build(best_hps)
history = final_model.fit(
    x_train_full, y_train_full,
    epochs=250,
    batch_size=32,
    verbose=1
)

# Evaluate the final model on the test data
predictions = final_model.predict(X_test)

predictions_unscaled = manual_inverse_transform(predictions, close_min, close_max)
y_test_unscaled = manual_inverse_transform(y_test.reshape(-1, 1), close_min, close_max)

# Calculate metrics on unscaled data
final_mae_unscaled = mean_absolute_error(y_test_unscaled, predictions_unscaled)
final_rmse_unscaled = np.sqrt(mean_squared_error(y_test_unscaled, predictions_unscaled))
final_mape_unscaled = tf.keras.losses.MAPE(y_test_unscaled, predictions_unscaled).numpy()

# Print the unscaled metrics
print('Final Test Unscaled Mean Absolute Error:', final_mae_unscaled)
print('Final Test Unscaled Root Mean Squared Error:', final_rmse_unscaled)
print('Final Test Unscaled Mean Absolute Percentage Error:', np.mean(final_mape_unscaled))

# Save the final model
final_model.save('best_LSTM3DREG.keras')

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/250
98/98 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0429 - mae: 0.1079 - mean_absolute_percentage_error: 83074.5078
Epoch 2/250
98/98 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 9.0751e-04 - mae: 0.0190 - mean_absolute_percentage_error: 6243.3428
Epoch 3/250
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 6.1768e-04 - mae: 0.0169 - mean_absolute_percentage_error: 5649.3706
Epoch 4/250
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 4.5056e-04 - mae: 0.0144 - mean_absolute_percentage_error: 3131.7341
Epoch 5/250
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 3.7374e-04 - mae: 0.0128 - mean_absolute_percentage_error: 1862.9266
Epoch 6/250
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 3.3478e-04 - mae: 0.0119 - mean_absolute_percentage_error: 1507.0984
Epoch 7/250
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 3.0939e-04 - mae: 0.0113 - mean_absolute_percentage_error: 2039.5563
Epoch 8/250
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 2.9179e-04 - mae: 0.0108 - mean_absolute_pe